### Checking shapes

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import rasterio

In [ ]:
crism_img_path = './raw_data/frt00005850_07_if167j_mtr3.img'
output = './raw_data/frt00005850_07_if167j_mtr3.npy'

with rasterio.open(crism_img_path) as src:
    data = src.read()

    data = np.transpose(data, (1, 2, 0)) # reorder dimensions, so that *number of bands* is the last one

    print(f"Data shape: {data.shape}")

    np.save(output, data)

In [ ]:
npy_data = np.load(output)
print(npy_data.shape)

### Preparing table with CRISM wavelength information

In [ ]:
tab_file_path = './raw_data/frt00005850_07_wv167j_mtr3.tab'

wavelength_table = np.loadtxt(tab_file_path, delimiter=',')

output_tab = './raw_data/CRISM_bands.npy'
np.save(output_tab, wavelength_table)

### EVC creation

In [ ]:
from create_evc_util import *

src_path = './raw_data/frt00005850_07_if167j_mtr3.hdr'
output = './frt00005850_07_ifevcj_mtr3.hdr'
bands = [24, 18, 1]

create_evc(src_path, output, bands)

### Batch EVC creation

In [ ]:
def modify_filename(filename):
    # Split the filename into segments by '_'
    parts = filename.split("_")

    # Ensure there are at least three segments
    if len(parts) < 3:
        raise ValueError("Filename does not have enough segments to process.")

    # Process the third segment
    third_segment = parts[2]
    modified_segment = f"{third_segment[:2]}evc{third_segment[-1]}"

    # Replace the third segment with the modified one
    parts[2] = modified_segment

    # Reassemble the filename
    return "_".join(parts)

In [ ]:
import os
from create_evc_util import *

input_folder = "" # raw CRISM .img and their corresponding .hdr files
output_folder = ""
bands = [24, 18, 1]

os.makedirs(output_folder, exist_ok=True)

for file in os.listdir(input_folder):
    try:
        if file.endswith(".hdr"):
            
            base_name = os.path.splitext(file)[0]
            input_path = os.path.join(input_folder, file)
            output_path = os.path.join(output_folder, f"{modify_filename(base_name)}.hdr")

            create_evc(input_path, output_path, bands)

            print(f"Finished: {file}")

    except Exception as e:
        print(f"Error processing {file}: {e}")